Evaluation, Original Kinetic-400 dataset

With results saved

In [4]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def load_video(video_path, num_frames=8):
    frames = []
    container = av.open(video_path)
    
    for frame in container.decode(video=0):
        frames.append(frame.to_rgb().to_ndarray())
        if len(frames) == num_frames:
            break
    
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames)

def load_labels(label_file):
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def evaluate_performance(config):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    
    video_labels = load_labels(config['label_file'])
    
    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'model_name': config['model_name'],
        'sample_size': config.get('sample_size', 'all'),
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config  # 保存完整配置
    }
    
    video_files = [f for f in os.listdir(config['video_directory']) if f.endswith('.mp4')]
    
    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42)) 
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
    
    for video_file in tqdm(video_files, desc="Processing videos"):
        video_name = video_file.split('.')[0]

        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        video_path = os.path.join(config['video_directory'], video_file)
        
        try:
            frames = load_video(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)

            with torch.no_grad():
                outputs = model(**inputs)
                pred_label = outputs.logits.argmax(-1).cpu().numpy()[0]

            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            
        except Exception as e:
            print(f"Error processing {video_file}: {str(e)}")
            continue

    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    
    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions'])
    }
    
    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)

    experiment_name = f"{config.get('experiment_name', 'experiment')}_{detailed_results['timestamp']}"
    result_file = save_dir / f"{experiment_name}.json"
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    print(f"\nResults saved to: {result_file}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_directory': '/home/z/Music/st/Kinetics-400/RQ1/videos_val',
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',
        'sample_size': 10000,  
        'random_seed': 42,  
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ1/step1/resultsoriginal',
        'experiment_name': 'timesformer_original'  
    }
    
    results = evaluate_performance(config)

/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing videos: 100%|██████████| 10000/10000 [31:05<00:00,  5.36it/s]


Performance Metrics:
Total samples processed: 10000
Accuracy: 0.5431
Precision: 0.6031
Recall: 0.5431
F1 Score: 0.5462

Results saved to: /home/z/Music/st/Kinetics-400/RQ1/step1/resultsoriginal/timesformer_original_20241229_194503.json


Accuracy: 0.5431
Precision: 0.6031
Recall: 0.5431
F1 Score: 0.5715

GaussianNoise

In [5]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def load_video(video_path, num_frames=8):
    frames = []
    container = av.open(video_path)
    
    for frame in container.decode(video=0):
        frames.append(frame.to_rgb().to_ndarray())
        if len(frames) == num_frames:
            break
    
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames)

def load_labels(label_file):
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def evaluate_performance(config):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    
    video_labels = load_labels(config['label_file'])
    
    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'model_name': config['model_name'],
        'sample_size': config.get('sample_size', 'all'),
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config 
    }
    

    video_files = [f for f in os.listdir(config['video_directory']) if f.endswith('.mp4')]
    
    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42))  
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
    
    for video_file in tqdm(video_files, desc="Processing videos"):
        video_name = video_file.split('.')[0]
        
        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        video_path = os.path.join(config['video_directory'], video_file)
        
        try:
            frames = load_video(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
                pred_label = outputs.logits.argmax(-1).cpu().numpy()[0]
  
            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            
        except Exception as e:
            print(f"Error processing {video_file}: {str(e)}")
            continue
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    
    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions'])
    }
    
    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    experiment_name = f"{config.get('experiment_name', 'experiment')}_{detailed_results['timestamp']}"
    result_file = save_dir / f"{experiment_name}.json"
    
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    print(f"\nResults saved to: {result_file}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_directory': '/home/z/Music/st/Kinetics-400/RQ1/gaussian_noise',
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',
        'sample_size': 10000,  
        'random_seed': 42,   
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ1/step1/resultsgaussian_noise',
        'experiment_name': 'timesformergaussian_noise' 
    }
    
    results = evaluate_performance(config)

/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing videos: 100%|██████████| 10000/10000 [31:07<00:00,  5.35it/s]



Performance Metrics:
Total samples processed: 10000
Accuracy: 0.4846
Precision: 0.5725
Recall: 0.4846
F1 Score: 0.4907

Results saved to: /home/z/Music/st/Kinetics-400/RQ1/step1/resultsgaussian_noise/timesformergaussian_noise_20241229_201613.json


Motion blur

In [6]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def load_video(video_path, num_frames=8):
    frames = []
    container = av.open(video_path)
    
    for frame in container.decode(video=0):
        frames.append(frame.to_rgb().to_ndarray())
        if len(frames) == num_frames:
            break
    
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames)

def load_labels(label_file):
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def evaluate_performance(config):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    
    video_labels = load_labels(config['label_file'])
    
    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'model_name': config['model_name'],
        'sample_size': config.get('sample_size', 'all'),
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config  
    }
    
    video_files = [f for f in os.listdir(config['video_directory']) if f.endswith('.mp4')]
    
    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42)) 
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
    
    for video_file in tqdm(video_files, desc="Processing videos"):
        video_name = video_file.split('.')[0]
        
        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        video_path = os.path.join(config['video_directory'], video_file)
        
        try:
            frames = load_video(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
                pred_label = outputs.logits.argmax(-1).cpu().numpy()[0]
            
            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            
        except Exception as e:
            print(f"Error processing {video_file}: {str(e)}")
            continue
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    
    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions'])
    }
    
    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    experiment_name = f"{config.get('experiment_name', 'experiment')}_{detailed_results['timestamp']}"
    result_file = save_dir / f"{experiment_name}.json"
    
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    print(f"\nResults saved to: {result_file}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_directory': '/home/z/Music/st/Kinetics-400/RQ1/motion_blur',
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',
        'sample_size': 10000
        'random_seed': 42,  
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ1/step1/resultsmotion_blur',
        'experiment_name': 'timesformer_motion_blur'  
    }
    
    results = evaluate_performance(config)

/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing videos: 100%|██████████| 10000/10000 [29:25<00:00,  5.66it/s]


Performance Metrics:
Total samples processed: 10000
Accuracy: 0.5225
Precision: 0.5928
Recall: 0.5225
F1 Score: 0.5277

Results saved to: /home/z/Music/st/Kinetics-400/RQ1/step1/resultsmotion_blur/timesformer_motion_blur_20241229_204722.json



/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Shotnoise

In [7]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def load_video(video_path, num_frames=8):
    frames = []
    container = av.open(video_path)
    
    for frame in container.decode(video=0):
        frames.append(frame.to_rgb().to_ndarray())
        if len(frames) == num_frames:
            break
    

    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames)

def load_labels(label_file):
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def evaluate_performance(config):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    

    video_labels = load_labels(config['label_file'])
    

    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'model_name': config['model_name'],
        'sample_size': config.get('sample_size', 'all'),
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config  # 保存完整配置
    }
    

    video_files = [f for f in os.listdir(config['video_directory']) if f.endswith('.mp4')]
    

    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42))  
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
    
    for video_file in tqdm(video_files, desc="Processing videos"):
        video_name = video_file.split('.')[0]
        
        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        video_path = os.path.join(config['video_directory'], video_file)
        
        try:
            frames = load_video(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)

            with torch.no_grad():
                outputs = model(**inputs)
                pred_label = outputs.logits.argmax(-1).cpu().numpy()[0]
            
            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            
        except Exception as e:
            print(f"Error processing {video_file}: {str(e)}")
            continue
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    
    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions'])
    }
    

    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    experiment_name = f"{config.get('experiment_name', 'experiment')}_{detailed_results['timestamp']}"
    result_file = save_dir / f"{experiment_name}.json"
    
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    print(f"\nResults saved to: {result_file}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_directory': '/home/z/Music/st/Kinetics-400/RQ1/shot_noise',
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',
        'sample_size': 10000,  
        'random_seed': 42,   
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ1/step1/resultsshot_noise',
        'experiment_name': 'timesformer_shot_noise' 
    }
    
    results = evaluate_performance(config)

/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing videos:   1%|          | 75/10000 [00:13<29:10,  5.67it/s]Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found
Processing videos:   1%|          | 77/10000 [00:13<22:33,  7.33it/s]

Error processing YS2xzcbG-T4.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/shot_noise/YS2xzcbG-T4.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found


Processing videos:   3%|▎         | 339/10000 [00:59<27:40,  5.82it/s]Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found
Processing videos:   3%|▎         | 341/10000 [00:59<21:53,  7.35it/s]

Error processing qi5Jtwx066I.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/shot_noise/qi5Jtwx066I.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found


Processing videos:  19%|█▉        | 1898/10000 [05:34<22:57,  5.88it/s]Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found


...

Error processing t3_UYvz8rdg.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/shot_noise/t3_UYvz8rdg.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found


Processing videos:  78%|███████▊  | 7772/10000 [22:52<06:48,  5.45it/s]Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found
Processing videos:  78%|███████▊  | 7774/10000 [22:52<05:02,  7.35it/s]

Error processing 51C5SqY-gTk.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/shot_noise/51C5SqY-gTk.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found


Processing videos: 100%|██████████| 10000/10000 [29:25<00:00,  5.66it/s]


Performance Metrics:
Total samples processed: 9994
Accuracy: 0.5438
Precision: 0.6054
Recall: 0.5438
F1 Score: 0.5453

Results saved to: /home/z/Music/st/Kinetics-400/RQ1/step1/resultsshot_noise/timesformer_shot_noise_20241229_211648.json


zoom blur

In [8]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def load_video(video_path, num_frames=8):
    frames = []
    container = av.open(video_path)
    
    for frame in container.decode(video=0):
        frames.append(frame.to_rgb().to_ndarray())
        if len(frames) == num_frames:
            break
    
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames)

def load_labels(label_file):
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def evaluate_performance(config):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    
    video_labels = load_labels(config['label_file'])

    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'model_name': config['model_name'],
        'sample_size': config.get('sample_size', 'all'),
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config  # 保存完整配置
    }
    

    video_files = [f for f in os.listdir(config['video_directory']) if f.endswith('.mp4')]
    
    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42))  
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
    
    for video_file in tqdm(video_files, desc="Processing videos"):
        video_name = video_file.split('.')[0]
        
        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        video_path = os.path.join(config['video_directory'], video_file)
        
        try:
            frames = load_video(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
                pred_label = outputs.logits.argmax(-1).cpu().numpy()[0]
            
            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            
        except Exception as e:
            print(f"Error processing {video_file}: {str(e)}")
            continue
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    

    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions'])
    }
    
    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    experiment_name = f"{config.get('experiment_name', 'experiment')}_{detailed_results['timestamp']}"
    result_file = save_dir / f"{experiment_name}.json"
    
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    print(f"\nResults saved to: {result_file}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_directory': '/home/z/Music/st/Kinetics-400/RQ1/zoom_blur',
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',
        'sample_size': 10000,  
        'random_seed': 42,    
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ1/step1/resultszoom_blur',
        'experiment_name': 'timesformer_zoom_blur'  
    }
    
    results = evaluate_performance(config)

/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing videos: 100%|██████████| 10000/10000 [29:39<00:00,  5.62it/s]


Performance Metrics:
Total samples processed: 10000
Accuracy: 0.4558
Precision: 0.5389
Recall: 0.4558
F1 Score: 0.4636

Results saved to: /home/z/Music/st/Kinetics-400/RQ1/step1/resultszoom_blur/timesformer_zoom_blur_20241229_214615.json
